# Exercise session 1: Overview of the Xpress Python Interface

## Creating a simple LP

First, create the simple mixed integer linear program:

$$
\begin{array}{lll}
  \min & x + y\\
  \textrm{s.t.} & 2x + 3y \ge 6\\
                & 4x + 2y \ge 7\\
                & x,y \ge 0
\end{array}
$$

In [26]:
# Location of manual and examples can be found by calling appropriate functions:

import xpress
print(xpress.manual())
print(xpress.examples())

/usr/local/lib/python3.7/site-packages/xpress/doc/python-interface.pdf
/usr/local/lib/python3.7/site-packages/xpress/examples/


In [40]:
import xpress as xp

# TODO Declare variables: x = xp.var(...)
x = xp.var('x',lb=-xp.infinity)
y = xp.var('y',lb=-xp.infinity)
# TODO Define constraints: constr1 = 2*x + ...
constr1 = x+2*y>= 5
constr2 = 4*x+y>=6
# TODO Define objective: obj = ...
objective = x+y

# TODO Create problem: p = xp.problem(...)
p = xp.problem(x,y,constr1, constr2, objective)

p.solve()

print(f'solution: x={p.getSolution(x)}, y={p.getSolution(y)}')


FICO Xpress v8.9.0, Hyper, solve started 6:48:38, Sep 19, 2020
Heap usage: 331KB (peak 331KB, 623KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Minimizing LP noname
Original problem has:
         2 rows            2 cols            4 elements
Presolved problem has:
         0 rows            0 cols            0 elements
Presolve finished in 0 seconds
Heap usage: 331KB (peak 343KB, 625KB system)
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0          3.000000      D      0     0        .000000     0
Uncrunching matrix
Optimal solution found
Dual solved problem
  0 simplex iterations in 0s

Final objective                       : 3.000000000000000e+00
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0
solution: x=1.0, y=2.0


Add the constraint $x + 2y \le 2$ to the problem, then re-solve. The problem should now be infeasible.

In [41]:
# TODO Create constraint
constr3 = x+2*y<=2

# TODO Add constraint to problem
p = xp.problem(x,y,constr1, constr2, constr3, objective)

# TODO Call solve() again
p.solve()


FICO Xpress v8.9.0, Hyper, solve started 6:48:39, Sep 19, 2020
Heap usage: 331KB (peak 331KB, 625KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Minimizing LP noname
Original problem has:
         3 rows            2 cols            6 elements
Problem is infeasible due to conflict between rows: R21 and R23
 
 
The problem is infeasible due to row R21
Presolve finished in 0 seconds
Heap usage: 330KB (peak 343KB, 627KB system)


Use the `iisall` function to understand what made the problem infeasible. If needed, type `help(p.iisall)` for its docstring.

In [42]:
p.iisall()

Size of IIS approximation is 2 rows and 2 columns
   Name      Type    Sense    Bound 
 R21         row     GE      5.000000
 R23         row     LE      2.000000
Found IIS 1 (2 rows and 0 bounds)
 
No more IIS can be found


Now delete the constraint (using the `problem.delConstraint` method) and, just to avoid the same solve as before, change the objective function coefficient of x to 4. Then re-solve and print primal and dual solution using `problem.getSolution` for the primal solution and `problem.getDual` for the dual solution. Recall that these functions can be called without arguments to obtain the whole vector, or with a variable/constraint object to get one or more values.

In [43]:
# TODO Use delConstraint() to delete the last added constraint
p.delConstraint(constr3)
# TODO Use p.chgobj() to change one objective coefficient.
# Its syntax has a list of variables for which the
# coefficient is changed and a list of corresponding 
# new coefficient, so the call is like p.chgobj([x1, x2, ...], [c1, c2, ...])
p.chgobj([x],[4])

p.solve()
print(f'Solution: x={p.getSolution(x)}, y={p.getSolution(y)}')
print(f'Dual: constr1->{p.getDual(constr1)}, constr2->{p.getDual(constr2)}')

FICO Xpress v8.9.0, Hyper, solve started 6:48:42, Sep 19, 2020
Heap usage: 4950KB (peak 4950KB, 861KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Minimizing LP noname
Original problem has:
         2 rows            2 cols            4 elements
Presolved problem has:
         0 rows            0 cols            0 elements
Presolve finished in 0 seconds
Heap usage: 4950KB (peak 4950KB, 861KB system)
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0          6.000000      D      0     0        .000000     0
Uncrunching matrix
Optimal solution found
Dual solved problem
  0 simplex iterations in 0s

Final objective                       : 6.000000000000000e+00
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0
Solution: x=1.0, y=2.0
Dual: constr1->-0.0, constr2->1.0


## Knapsack problem

Formulate and solve a knapsack problem with the following value/weight vectors:

$$
\begin{array}{lllrrrrrrr}
v& =& (12,&15,& 9,&11,& 8,& 7,&5)\\
w& =& (13, &18,& 9,& 12,& 8,& 10,& 4)
\end{array}
$$

and with knapsack capacity $C=40$. You will need the `xpress.Sum` operator.

In [46]:
import xpress as xp

v = [12,15,9,11,8, 7,5]
w = [13,18,9,12,8,10,4]
C = 40

# Number of variables
n = len(v)

# TODO Define a list of n binary variables
x = [xp.var(f"{i}",vartype=xp.binary) for i in range(n)] 

# TODO Define the knapsack constraint and its objective
con = xp.Sum(x[i] * w[i] for i in range(n)) <= C
obj = xp.Sum(x[i] * v[i] for i in range(n))

# TODO Create a problem and add variable list, constraint, and objective
p = xp.problem()
p.addVariable(x)
p.addConstraint(con)
p.setObjective(obj, sense=xp.maximize)


p.solve()

print(p.getSolution())


FICO Xpress v8.9.0, Hyper, solve started 6:50:11, Sep 19, 2020
Heap usage: 331KB (peak 331KB, 598KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Maximizing MILP noname
Original problem has:
         1 rows            7 cols            7 elements         7 globals
Presolved problem has:
         1 rows            7 cols            7 elements         7 globals
Presolve finished in 0 seconds
Heap usage: 357KB (peak 361KB, 600KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 4.00e+00,  1.80e+01] / [ 2.50e-01,  1.12e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  4.00e+01] / [ 1.00e+00,  2.50e+00]
  Objective      [min,max] : [ 5.00e+00,  1.50e+01] / [ 5.00e+00,  1.50e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 1.3GB
 *** Heuristic solution found:      .000000      Time: 0 ***
 *** Heuristic solution found: 

## Reading and solving a MIP

Let's try to solve something bigger, like a file from the MIPLIB 2017 collection. Specifically, `roi2alpha3n4`. It should be available in the directory with this exercise session; load it and solve it as a Python problem.

In [48]:
# TO BE DONE IN CLASS

filename = 'mipinst.mps.gz'
p = xp.problem()
p.read(filename)
p.controls.maxtime = -30
p.solve()


Reading Problem roi2alpha3n4
Problem Statistics
        1251 (      1 spare) rows
        6816 (      0 spare) structural columns
      878812 (    174 spare) non-zero elements
Global Statistics
        6642 entities        0 sets        0 set members
FICO Xpress v8.9.0, Hyper, solve started 7:12:52, Sep 19, 2020
Heap usage: 12MB (peak 12MB, 858KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Minimizing MILP roi2alpha3n4
Original problem has:
      1251 rows         6816 cols       878812 elements      6642 globals
Presolved problem has:
       443 rows         5364 cols       857240 elements      5190 globals
LP relaxation tightened
Presolve finished in 2 seconds
Heap usage: 34MB (peak 50MB, 860KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e-06,  4.00e+00] / [ 1.00e-06,  1.00e+00]
  RHS and bounds [min,max] : [ 5.70e-01,  4.00e+00] / [ 5.70e-01,  4.0

Add a new constraint: the norm of the variable vector is at most 10. Even if the problem was read from a file, you can get the vector of all variables with `problem.getVariable`.

Note that before the constraint is added we'll have to postsolve the problem (using the `problem.postsolve()` function) as, for reasons of performance, the problem remains in a presolved state after the `solve()` call.

In [50]:
import numpy as np
p.postsolve()

# TODO: Get all variables as a numpy array (with appropriate numpy dtype) using the getVariable()
# function wrapped in a np.array() call, i.e., np.array(p.getVariable(...), dtype=xp.npvar)
v = np.array(p.getVariable(), dtype=xp.npvar)

# TODO: Add constraint with sum of squares of all variables at most 100 (i.e. norm <= 10)
p.addConstraint(xp.Dot(v,v) <= 100)

# Set a time limit and solve the problem
p.controls.maxtime=-30
p.solve()

Uncrunching matrix
FICO Xpress v8.9.0, Hyper, solve started 7:17:31, Sep 19, 2020
Heap usage: 21MB (peak 65MB, 1283KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Minimizing MIQCQP roi2alpha3n4
Original problem has:
      1252 rows         6816 cols       878812 elements      6642 globals
         1 qrows        6816 qrowelem
Converted 1 quadratic matrices to their separable equivalent
Converted 1 separable quadratic matrices to rotated cones
Presolved problem has:
      7261 rows        13635 cols       884221 elements      6642 globals
      6817 cones       13634 celems
LP relaxation tightened
Presolve finished in 8 seconds
Heap usage: 49MB (peak 93MB, 1283KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e-06,  4.00e+00] / [ 1.00e-06,  1.00e+00]
  RHS and bounds [min,max] : [ 5.70e-01,  1.00e+02] / [ 5.00e-01,  1.00e+02]
  Objective      [min,max] : [

With a real MIP at hand, how about we try to see the structure of the coefficient structure from a graphical standpoint? Use `matplotlib` to draw a density chart.

In [51]:
help(p.getrows)

Help on built-in function getrows:

getrows(...) method of xpress.problem instance
    Purpose: 
    Returns the nonzeros in the constraint matrix for the rows in a given
    range.
    
    Synopsis:
    
    problem.getrows(mstart, mclind, dmatval, size, first, last)
    
    Arguments:
    mstart: 
     Array which will be filled with the indices indicating the starting offsets in the mclind and dmatval arrays for each requested row. It must be of length at least last-first+2. Column i starts at position mstart[i] in the mrwind and dmatval arrays, and has mstart[i+1]-mstart[i] elements in it. May be None if not required.
    mclind: 
     Arrays which will be filled with at most size column of the nonzero elements for each row. May be None if not required.
    dmatval: 
     Array  which will be filled with at most size nonzero element values. May be None if not required.
    size: 
     Maximum number of elements to be retrieved.
    first: 
     First row in the range.
    last: 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sparse

p.postsolve()
mstart, mclind, dmatval = [], [], []
p.getrows(mstart, mclind, dmatval, p.attributes.elems, 0, p.attributes.rows - 1)

# Extract all coefficient data with getrows()

# mclind contains variable objects, not indices.
# Extract the index for all of them with getIndex()

# Use the spy() function for plotting the matrix
plt.show()

### NumPy operators and arrays of variables/constraints

NumPy is an essential toolbox for Python users. The Xpress Python interface can handle (multi-)arrays of floats, variables, expressions, constraints as naturally as with lists thereof.

In [52]:
import numpy as np

vars1 = xp.vars(20,      vartype=xp.binary, name='x')

We can also rewrite the knapsack problem using NumPy constructs.

In [65]:
import xpress as xp
import numpy as np

v = np.array([12,15,9,11,8, 7,5])
w = np.array([13,18,9,12,8,10,4])
C = 40

n = len(v)


# Create numpy array of n binary variables
alist = np.array([xp.var(f"{i}",vartype=xp.binary) for i in range(n)])

# Create constraints and objective for the knapsack
con = xp.Sum(alist*w) <= C
obj = xp.Sum(alist*v)
p = xp.problem()
p.addVariable(alist)
p.addConstraint(con)

# Create (maximization!) problem
p.setObjective(obj, sense=xp.maximize)
p.solve()

print(p.getSolution())


FICO Xpress v8.9.0, Hyper, solve started 11:10:42, Sep 20, 2020
Heap usage: 331KB (peak 331KB, 1029KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Maximizing MILP noname
Original problem has:
         1 rows            7 cols            7 elements         7 globals
Presolved problem has:
         1 rows            7 cols            7 elements         7 globals
Presolve finished in 0 seconds
Heap usage: 357KB (peak 361KB, 1031KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 4.00e+00,  1.80e+01] / [ 2.50e-01,  1.12e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  4.00e+01] / [ 1.00e+00,  2.50e+00]
  Objective      [min,max] : [ 5.00e+00,  1.50e+01] / [ 5.00e+00,  1.50e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 1.3GB
 *** Heuristic solution found:      .000000      Time: 0 ***
 *** Heuristic solution foun

**Exercise**: Create a random 20x30 numpy matrix $A$ and vectors ${\bf b}$ and ${\bf c}$, then create a problem with ${\bf c}$ as objective function coefficient vector and constraints $A{\bf x} \le {\bf b}$.

In [66]:
n = 3 # number of variables
m = 2 # number of constraints

# TODO Create numpy random matrix A, b, and c using the function
# np.random.random(), with a single parameter k to create a
# 1-dimensional array and the tuple (k1,k2) for a matrix
A = np.random.random((m,n))
b = np.random.random(m)
c = np.random.random(n)

# TODO Create a variable vector
x = xp.vars(n)

# TODO Create objective and constraint using the xpress.Dot() operator.
obj = xp.Dot(c, x)
con = xp.Dot(A, x) <= b

# TODO Create problem
p = xp.problem(x, obj, con, sense=xp.maximize)

p.solve()


FICO Xpress v8.9.0, Hyper, solve started 11:10:43, Sep 20, 2020
Heap usage: 331KB (peak 331KB, 1032KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Maximizing LP noname
Original problem has:
         2 rows            3 cols            6 elements
Presolved problem has:
         2 rows            3 cols            6 elements
Presolve finished in 0 seconds
Heap usage: 331KB (peak 343KB, 1033KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 3.96e-01,  8.37e-01] / [ 3.96e-01,  8.37e-01]
  RHS and bounds [min,max] : [ 3.75e-01,  9.55e-01] / [ 3.75e-01,  9.55e-01]
  Objective      [min,max] : [ 3.86e-01,  8.84e-01] / [ 3.86e-01,  8.84e-01]
Autoscaling applied standard scaling

 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0          1.068023      D      2     0        .000000     0
     1           .495409      D      0     0        .0000

Let's spice it up: make the objective function quadratic, with a matrix $Q$ that is PSD but not necessarily diagonal. Make the objective

$$
(x-x_0)^T Q (x-x_0)
$$

In [47]:
# This matrix is positive semidefinite, so that the problem is convex.
Q = np.random.random((n,n)) - 0.5 + 3*n*np.eye(n)

# Creates a uniformly random point with coordinates in [0,10]
x0 = 10 * np.random.random(n)  

# TODO Alter the objective function so as to minimize weighted
# (with Q) distance between x and x0: (x-x0)' Q (x-x0)

p.solve()

FICO Xpress v8.9.0, Hyper, solve started 6:50:54, Sep 19, 2020
Heap usage: 2599KB (peak 3064KB, 601KB system)
Detected container-enforced core limit of 1
Detected container-enforced memory limit of 2048 MB
Maximizing MILP noname
Original problem has:
         1 rows            7 cols            7 elements         7 globals
Presolved problem has:
         1 rows            7 cols            7 elements         7 globals
Presolve finished in 0 seconds
Heap usage: 2625KB (peak 3064KB, 601KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 4.00e+00,  1.80e+01] / [ 2.50e-01,  1.12e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  4.00e+01] / [ 1.00e+00,  2.50e+00]
  Objective      [min,max] : [ 5.00e+00,  1.50e+01] / [ 5.00e+00,  1.50e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 1.3GB
 *** Heuristic solution found:      .000000      Time: 0 ***
 *** Heuristic solution foun

Obviously we can use arrays of variables in constraints. Let's consider the following problem:

### Production planning with quadratic cost

A factory must plan production on one machine for the upcoming $N=12$ days. If, on one day, the machine is on, it must pay a fixed cost F, regardless of how much is produced on that machine. If the machine is on, the production must range between L and U. It must also pay a cost that is C times the square of the produced amount.

The demand at a given day is $w_i$, and the unsold product can be stored in a storage container of capacity $M$. Finally, the storage level at the end of the $N$ days must be the same as the beginning, which is set as $S$.

**Model**. Define two classes of variables, $x$ (continuous) and $y$ (binary), indicating the amount of production in a given day and whether the machine is on or off, respectively. Also, introduce a storage variable $s$ that is also indexed by the set of periods.

In [63]:
import xpress as xp

n = 12  # number of periods
N = range(n)

# Input parameters

C = 2   # proportional cost
F = 9000  # fixed cost
L = 10   # min. production level
U = 90   # max. production level
M = 170  # maximum storage level
S = 25   # initial storage level

# Demand
w = [60, 15, 25, 70, 70, 85, 10, 5, 65, 40, 50, 15]

# TODO Create variables: y is a binary vector of size n, x has size
# n, while s has size n+1 to include storage at time n + 1
x = [xp.var() for _ in N]
y = [xp.var(vartype=xp.binary) for _ in N]
s = [xp.var(ub=M) for _ in range(n + 1)]


# TODO Total production cost (to be minimized)
objective = xp.Sum(F*y[i] + C * (y[i]*x[i]**2) for i in N)

# TODO y determines bounds on x: if y=1, then x is
# between L and U, otherwise it is 0. How to do it
# using notions learned during the course? Hint: think
# about big-M.
x_lb = [x[i] >= L*y[i] for i in N]
x_ub = [x[i] <= U*y[i] for i in N]

# TODO Constrain storage at last period
store_end = s[-1] == S
store_ini = s[0] == S

# TODO Production balance:
# the storage at time i plus the production at time i
# must equal
# the storage at time i+1 plus the demand at time i.
balance = [s[i] + x[i] == s[i+1] + w[i] for i in N]

# TODO create a problem with the variables, objectives, and constraints
# as defined above.

p = xp.problem(
    x, y, s,
    x_lb, x_ub, store_end, store_ini, balance,
    objective
)

p.solve()

print('time demand machine productn storage')
for i in N:
    print('{0:4d} {1:6d} {2:7.0f} {3:8.4f} {4:7.4f}'.format(i, w[i],
                                                            p.getSolution(y[i]),
                                                            p.getSolution(x[i]),
                                                            p.getSolution(s[i])))


FICO Xpress v8.9.0, Hyper, solve started 11:02:36, Sep 20, 2020
Original problem size
   linear:    39 rows, 39 columns, 87 linear coefficients
   nonlinear: 1 coefficients, 95 tokens
Nonlinear presolve
   compressing formula space (in use\total : 96\713)
   removed 22 formulas and 616 formula tokens
   objective transfer row objtransferrow through variable objtransfercol
   bound tightening reduced 20 bounds
Presolved problem size
   linear:    39 rows, 39 columns, 87 linear coefficients
   nonlinear: 1 coefficients, 95 tokens
Problem is nonlinear presolved
Maximum expanded nl-formula size: 95  (row 'objtransferrow')
Total tokens: 95
  32  parallel calculation threads
  Jacobian: symbolic differentiation
           1 base AD formula, 96 average complexity
          24 in the Jacobian, 6 average complexity
Initial point objective:  157112.50
Absolute / relative validation:          24.000  /    24.000
Minimizing problem using Xpress-SLP
Xpress-SLP Augmentation Statistics:
  Columns:
 

**Homework**: Rewrite the problem using NumPy for defining parameters, variables, constraints, and objective.

In [64]:
import xpress as xp
import numpy as np

# Demand is translated to a NumPy array for using NumPy operators
w = np.array(w)

# TODO Variables: use vars() but remember to name them in order to
# avoid variables with same name. For instance, use x = xp.vars(..., name='prod')
# for production variables
x = xp.vars(n)
y = xp.vars(n,vartype=xp.binary)
s = xp.vars(n+1, ub=M)

# TODO Total production cost (to be minimized)
objective = xp.Sum(F*y + C * (y*x**2) )

# TODO y determines bounds on x (just two numpy inequalities)
x_lb = x >= L*y
x_ub = x <= U*y

# TODO storage at last period (doesn't change from previous model)
store_end = s[-1] == S
store_ini = s[0] == S

# TODO Production balance: use the range operator of arrays for the s[] variable to write a 
# single numpy constraint.
balance = s[:n] + x[:] == s[1:] + w

# TODO create problem

p.solve()

print('time demand machine productn storage')
for i in N:
    print('{0:4d} {1:6d} {2:7.0f} {3:8.4f} {4:7.4f}'.format(i, w[i],
                                                            p.getSolution(y[i]),
                                                            p.getSolution(x[i]),
                                                            p.getSolution(s[i])))

FICO Xpress v8.9.0, Hyper, solve started 11:07:20, Sep 20, 2020
Original problem size
   linear:    39 rows, 39 columns, 87 linear coefficients
   nonlinear: 1 coefficients, 95 tokens
Nonlinear presolve
   compressing formula space (in use\total : 96\337)
   removed 24 formulas and 240 formula tokens
   objective transfer row objtransferrow through variable objtransfercol
   bound tightening reduced 20 bounds
Presolved problem size
   linear:    39 rows, 39 columns, 87 linear coefficients
   nonlinear: 1 coefficients, 95 tokens
Problem is nonlinear presolved
Maximum expanded nl-formula size: 95  (row 'objtransferrow')
Total tokens: 95
  32  parallel calculation threads
  Jacobian: symbolic differentiation
           1 base AD formula, 96 average complexity
          24 in the Jacobian, 6 average complexity
Initial point objective:  157112.50
Absolute / relative validation:          24.000  /    24.000
Minimizing problem using Xpress-SLP
Xpress-SLP Augmentation Statistics:
  Columns:
  

ModelError: Object does not belong to problem or cannot be identified within expression